# 原始数据加载（docs）

In [ ]:
# 实时检测外部函数更新
%load_ext autoreload
%autoreload 2 

In [ ]:
import os
# 1. 设置国内镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from bertopic import BERTopic
import torch # 新增：导入PyTorch（sentence_transformers的底层）
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import joblib
import time
import json
import pandas as pd 
from report_generator import generate_bertopic_report
import re

# Config

In [ ]:
# 版本控制
version = 'amb3'
start_time = 2000
end_time = 2025
data_source = 'GF专利'
model_name = "all-mpnet-base-v2" #短文本
# model_name = 'jinaai/jina-embeddings-v2-base-en' #长文本,GPU容易炸

year_range = f'{start_time}-{end_time}'

# 全量数据集的词嵌入结果保存位置，计算耗时长，一定要保存，避免重复计算，只有数据集需要增加或变更时才需要重新计算
embeddings_path = r'results\embedding_results\embeddings_patents_zf_amb_slide_window3.pkl' 
# HDBSCAN网格搜索范围设置
search_sizes = [15, 20, 30, 50, 70,100,150,200]
# 当主题数量非常庞大时，可以进一步层次聚类合并主题，n_groups 是你想要的分组数量
n_groups = 50

# --- UMAP 参数设置 ---
n_neighbors = 15
umap_params = {
    "n_neighbors": n_neighbors,
    "n_components": 5,
    "min_dist": 0.0,
    "metric": 'cosine',
    "random_state": 5,
    "low_memory": True
}

# --- HDBSCAN参数设置 ---
HDBSCAN_cfg = {
    "min_samples": 10,
    "metric": 'euclidean',
    "prediction_data": True
}

# --- vectorizer_model 参数设置 ---
vectorizer_params = {
    "ngram_range": (1, 3), # 词组范围：1-3个词
    "min_df": 20, # 过滤低频噪声，对大样本非常重要
    "max_features": 100000, # 防止内存溢出
    "max_df": 0.7 #过滤掉在超过 50% 文本中都出现的词。这些词通常是行业背景词,会干扰主题关键词的提取。
}

# --- AI识别系统提示词设计 ---
system_prompt = """你是一名专业的国防专利与技术分析专家。
你的任务是根据提供的主題关键词，为每一个主题确定概括性名称。
要求：
1. 使用中文回答。
2. 名称格式为"编号：主题名称", 
示例如下
0: "1.武器装备结构设计与制图", 
1: "2.半导体工艺与微电子器件"

3. 输出JSON格式的字典，鍵为主题编号(数据类型为整型int)，值为主题名称(string)。
4. 主题简明，不要有多余的解释。"""

任务说明（以文献数据处理为例）：
- 1.将EXCEL数据转换为json格式，按照3TI 、1AB和2KEY WORDS，完成文本的字符串拼接，json文件中需要`year`、`title`、`abstract`、`keywords`以及`combined_text`字段；（AI应该很容易写出来相应的转换代码，该文件作为原始主题建模的原始数据文件）；
- 2.变量`docs`是模型的输入，将json文件中的`combined_text`传入到该向量中形成一个列表型数据；
- 3.根据后续代码跑通这一套BERTopic技术路线

In [ ]:
# 从JSON文件加载数据
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# # 打印最后两条数据进行检查
# for item in data[-2:]:
#     print(item['year'])
#     #print(item['title'])
#     #print(item['abstract'])
#     print(item['combined_text'])

# 文本
docs_original = [item['combined_text'] for item in data]
timestamps = [item['year'] for item in data]

# 数据清洗
def preprocess_all(texts):
    cleaned_list = []
    for t in texts:
        # 移除特殊字符和多余空格
        t = re.sub(r"[^\w\s\.,;]", " ", t) # 只保留字母、数字、空格和几种标点
        t = re.sub(r"\s+", " ", t).strip() # 移除多余空格
        cleaned_list.append(t)
    return cleaned_list

# 运行
# 检查清洗效果
docs = preprocess_all(docs_original)
# 检查数据对齐
doc_count = len(docs)
time_count = len(timestamps)

# 如果数量对不上就报错，并打印出具体数值
assert doc_count == time_count, f"数据不匹配！文档有 {doc_count} 条，但时间戳有 {time_count} 个。"

print(f"数据检查通过，共 {doc_count} 条记录。")

In [ ]:
#print('共获取数据总量:', len(docs),'条')

# 创建模型

In [ ]:
# CPU同样可以运行，只是速度较慢
# --- 核心：检查并指定设备 ---
# 检查CUDA（即NVIDIA GPU支持）是否可用
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU可用，正在使用: {torch.cuda.get_device_name(0)}")
    # 可选：如果你想使用特定的GPU（如第0块）
    # device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("⚠️  未检测到可用GPU，将使用CPU。")
    import torch
print(torch.__version__) # 查看PyTorch版本
print(torch.cuda.is_available()) # 核心：查看CUDA是否可用

# 预训练模型加载

In [ ]:
# 1. 统一定义路径
local_model_path = f"./my_models/{model_name}"  # 固定路径

# 2. 检查并加载模型
if not os.path.exists(local_model_path):
    print(f"模型不存在，正在下载并保存到本地: {model_name}")
    # 直接从huggingface下载amb短文本嵌入模型
    sentence_model = SentenceTransformer(model_name)
    # # 直接从huggingface下载jinai长文本嵌入模型
    # sentence_model = SentenceTransformer(model_name,trust_remote_code=True)

    # 保存到本地
    sentence_model.save(local_model_path)
    print(f"模型已保存至：{local_model_path}")
else:
    print(f"模型已存在，从本地加载: {local_model_path}")
    # 从本地加载
    sentence_model = SentenceTransformer(local_model_path)
    #sentence_model = SentenceTransformer(local_model_path,trust_remote_code=True)
# sentence_model.max_seq_length = 2048

In [ ]:
from slide_window import sliding_window_encode
# 2. 计算并保存词向量
def compute_and_save_embeddings(docs, save_path='embeddings_test'):
    """计算并保存词向量"""
    if os.path.exists(save_path):
        print(f"加载已保存的嵌入: {save_path}")
        with open(save_path, 'rb') as f:
            embeddings = pickle.load(f)
    else:
        print("计算新的嵌入...")
        embeddings = sliding_window_encode(
            docs, 
            sentence_model, 
            window_size=512,
            stride=256,
            inference_batch_size=32
            )



        # embeddings = sentence_model.encode(docs,
        #                                 batch_size=2,
        #                                 show_progress_bar=True,
        #                                 convert_to_numpy=True # 显式转为 numpy 释放 GPU 空间
        #                                 )
        with open(save_path, 'wb') as f:
            pickle.dump(embeddings, f)
        print(f"嵌入已保存到: {save_path}")
    return embeddings



In [ ]:
def load_stopwords(file_path):
    """从JSON文件加载停用词列表"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = json.load(f)
        print(f"已加载 {len(stopwords['common'])} 个停用词")
        return stopwords['common']  # 转换为集合提高查找效率
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在")
        return set()
    except json.JSONDecodeError:
        print(f"错误：文件 {file_path} 不是有效的JSON格式")
        return set()

# # 测试示例
# stopwords = load_stopwords('data\stopwords.json')
# print(stopwords)

# 📊 BERTopic 核心模型参数设置指南

在 BERTopic 中，**UMAP（降维）** 和 **HDBSCAN（聚类）** 是决定主题质量的两个关键环节。对于 12.5 万条的大规模长文本数据，合理的参数配置能平衡“细节捕捉”与“结构稳健性”。

---

## 一、 UMAP：将高维语义压缩至低维

**主要任务：** 把 BERT 提取的几百维向量降至 5 维，同时保留文本间的相似性。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`n_neighbors`** | **20** | **视野范围**。决定每个点看多远。值设为 20 能在保留局部细节的同时，兼顾 12.5 万条数据的整体分布，防止主题过于破碎。 |
| **`n_components`** | **5** | **保留维度**。降维后的“维度数”。设置为 5 比 2 或 3 能保留更多语义细节，避免信息在压缩过程中丢失。 |
| **`min_dist`** | **0.0** | **挤压程度**。点与点之间的最小间距。设置为 0.0 会让相似的文档尽可能“抱团”，有助于 HDBSCAN 发现更清晰的边界。 |
| **`metric`** | **'cosine'** | **相似度标准**。文本处理的黄金标准。它关注的是词汇的方向（语义），而不是文本的长短。 |
| **`random_state`** | **5** (或 42) | **结果保险单**。固定随机种子。确保你下次运行代码时，降维后的结果是一模一样的。 |

---

## 二、 HDBSCAN：在低维空间发现主题

**主要任务：** 在降维后的空间里寻找高密度区域，每个高密度区域就是一个“主题”。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`min_cluster_size`** | **50** | **成团门槛**。定义一个主题最少要包含多少篇文档。对于 12.5 万条数据，50 是一个较细的粒度；如果觉得主题太碎，可以调大至 200-500。 |
| **`min_samples`** | **3** | **严苛程度**。定义核心点的门槛。值越小（如 3），算法越“宽容”，会将更多边缘点纳入主题；值越大，噪声（-1）会越多。 |
| **`metric`** | **'euclidean'** | **测距方式**。由于 UMAP 已经处理过余弦相似度，降维后的 5 维空间通常直接使用欧几里得距离来划分簇。 |

---

## 三、 针对“降噪”与“粒度”的调优秘籍

如果你在运行后发现效果不理想，可以参考以下逻辑进行微调：

### 1. 噪声（-1）比例太高怎么办？

* **方法 A（调松）：** 保持 `min_samples=3` 甚至降为 **1**。这会强制让更多游离的文档归入最近的主题。
* **方法 B（后处理）：** 不动聚类模型，训练后使用 `topic_model.reduce_outliers()` 强制归类。

### 2. 主题太细碎、数量太多怎么办？

* **增加 `min_cluster_size**`：从 50 提高到 **300 或 500**。这是控制主题总数最直接的杠杆。
* **后期合并**：利用 `topic_model.reduce_topics(nr_topics=100)` 手动指定最终想要的主题数。

### 3. 主题之间分不开（语义重叠）？

* **调小 `n_neighbors**`：从 20 降到 **15 或 10**。这会迫使 UMAP 更加关注局部差异，让不同的簇分得更开。

---

> **💡 提示：**
> 对于大规模数据，建议先用这套参数跑一个样本集（如 2 万条），观察主题分布。一旦确定 UMAP 效果不错，请务必**保存 UMAP 结果**（`.npy` 或 `.joblib`），以节省后续反复调参的时间成本。

# BERTopic模型创建

In [ ]:
# 你的代码流程：
# +-------------------+       +-------------------------+       +---------------+
# | 你的预计算嵌入     |------>| BERTopic.fit_transform()|------>|最终主题结果   |
# | embeddings_patents|       |                         |       |               |
# | _zf_v1.pkl        |       |                         |       | topics, probs |
# +-------------------+       +-------------------------+       +---------------+
#                                    │
#                                    ▼ 内部处理流程
#                         +-----------------------+
#                         │ 1. 接收你的 embeddings │
#                         +-----------------------+
#                                    │
#                                    ▼
#                         +-----------------------+
#                         │ 2. ✅ UMAP 降维        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +-----------------------+
#                         │ 3. HDBSCAN 聚类        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +----------------------------------+
#                         │ 4. Vectorizer\c-TF-IDF提取主题词等│
#                         +----------------------------------+

## 词向量嵌入

In [ ]:
# 计算并保存词向量，全量词向量嵌入，建议保存，以免重复计算
embeddings = compute_and_save_embeddings(docs, embeddings_path)

## UMAP降维

In [ ]:
# 文件路径定义
umap_cache_path = f'umap_model_{data_source}_{start_time}-{end_time}_{version}.joblib'
umap_model = UMAP(**umap_params)
# 检查模型是否存在
if os.path.exists(umap_cache_path):
    print(f"🚀 检测到已存在的降维模型，正在直接加载: {umap_cache_path}")
    umap_embeddings = joblib.load(umap_cache_path)
else:
    print("⏳ 未发现降维模型，开始执行 UMAP 降维计算（大数据量耗时较长）...")
    # 确保 umap_params 已经定义
    umap_embeddings = umap_model.fit_transform(embeddings)
    
    # 保存结果
    joblib.dump(umap_embeddings, umap_cache_path)
    print(f"✔ UMAP 降维完成并已保存至: {umap_cache_path}")

# 接下来可以直接使用 umap_embeddings 进行 HDBSCAN 聚类

## HDBSCAN聚类（网格搜索）

In [ ]:
best_topics = None
best_m_size = None
best_score = float('inf')

search_history = [] # 新增：用于记录网格搜索过程

print(f"{'Size':<10} | {'主题数':<8} | {'负样本数':<10} | {'噪声比例':<10} | {'耗时':<8}")
print("-" * 65)

for m_size in search_sizes:
    start_t = time.time()
  
    clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=m_size)

    
    labels = clusterer.fit_predict(umap_embeddings)
    
    # 计算指标
    n_outliers = (labels == -1).sum() # 负样本数
    n_topics = len(set(labels)) - (1 if -1 in labels else 0)
    outlier_perc = n_outliers / len(labels)
    duration = time.time() - start_t

    # 保存历史记录
    res = {
        "min_cluster_size": m_size,
        "n_topics": n_topics,
        "n_outliers": n_outliers,
        "outlier_perc": f"{outlier_perc:.1%}",
        "duration": f"{duration:.1f}s",
        "is_best": False
    }
    search_history.append(res)

    # 筛选逻辑：选噪声最小的
    if outlier_perc < best_score:
        best_topics = n_topics
        best_score = outlier_perc
        best_m_size = m_size
    

    print(f"{m_size:<10} | {n_topics:<10} | {n_outliers:<12} | {outlier_perc:<12.1%} | {duration:<8.1f}s")
print("-" * 65)
if best_m_size is None:
    print("未在预设主题数范围内找到参数，建议调低 min_dist 或检查 UMAP 效果")
else:
    print(f"🏆 符合条件的负样本数最少的参数值: min_cluster_size = {best_m_size} (负样本比例: {best_score:.1%},主题数：{best_topics})")

# 标记最优参数
for item in search_history:
    if item["min_cluster_size"] == best_m_size:
        item["is_best"] = True



## 正式创建BERTopic模型

In [ ]:
## HDBSCAN聚类
# 使用 ** 进行字典解包
best_clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=best_m_size)
# 加载停用词
stop_words = load_stopwords('data\stopwords.json')
# 创建CountVectorizer模型,自定义停用词
# vectorizer_model = CountVectorizer(
#     ngram_range = (1,3), # 词组范围：1-3个词
#     stop_words = stop_words, # 停用词
#     min_df = 10, # 过滤低频噪声，对大样本非常重要
#     max_features=100000 # 防止内存溢出
#     )
vectorizer_model = CountVectorizer(
    **vectorizer_params,
    stop_words = stop_words
    )

# 初始化 BERTopic 时带上它
topic_model = BERTopic(
  embedding_model=sentence_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,       # 加上这一行
  hdbscan_model=best_clusterer # 加上这一行
)
#传入训练好的词向量，fit_transformer()包括了UMAP降维+HDBSCAN聚类，不建议这么做，可以将其拆开减少计算
#topics, probs = topic_model.fit_transform(docs, embeddings=embeddings,y = labels) 
#prob计算很花时间
topics, _ = topic_model.fit_transform(docs, embeddings=embeddings)
# 保存整个主题模型


save_path = rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}"
topic_model.save(save_path)
print(rf"主题模型已保存到 results\topic_models\bertopic_{data_source}_{year_range}_{version}")

In [ ]:
## 直接加载主题模型
topic_model = BERTopic.load(rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}")
print("初始主题模型加载成功！")

# 原始主题可视化

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
# 降至二维，再可视化
reduced_embeddings = UMAP(n_neighbors=n_neighbors, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [ ]:
import pandas as pd

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}数据Bertopic主题聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_Allset_{year_range}_{data_source}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_bertopic_report(umap_cfg = umap_params, 
                         HDBSCAN_cfg=HDBSCAN_cfg,
                         vectorizer_cfg = vectorizer_params,
                         history= search_history,
                         best_size = best_m_size,
                         model_name = model_name, 
                         output_path = report_name)
print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=50, # 只预览前50个主题，若需要所有主题的关键词，则设置为best_topics
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=80, r=80, t=100, b=80),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 hierarchy.html



topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

# 层次聚类

# 计算可以进一步合并的主题

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, cut_tree

# 1. 针对数据结构提取 Scipy 链接矩阵
def get_linkage_matrix_final(df):
    # 提取左子节点、右子节点、距离
    # 你的列名是 Child_Left_ID, Child_Right_ID, Distance
    left = df['Child_Left_ID'].values.astype(float)
    right = df['Child_Right_ID'].values.astype(float)
    dist = df['Distance'].values.astype(float)
    
    # 提取主题数量：由于你的 Topics 列存的是列表，我们需要计算每个列表的长度
    num_topics = df['Topics'].apply(len).values.astype(float)
    
    # 合并为 Scipy 需要的 Linkage Matrix (n-1, 4)
    return np.column_stack([left, right, dist, num_topics])

# 2. 执行矩阵转换
# 注意：Scipy 要求矩阵按距离从小到大排序，BERTopic 返回的顺序通常是反的（根节点在最前）
df_sorted = hierarchical_topics.sort_values('Distance')
linkage_matrix = get_linkage_matrix_final(df_sorted)

# 3. 进行分组

clusters = cut_tree(linkage_matrix, n_clusters=n_groups).flatten()

# 4. 映射回原始主题标签
# 获取所有原始主题 ID（排好序，确保与 cut_tree 结果对应）
all_topics = sorted([t for t in topic_model.get_topics().keys() if t != -1])

df_res = pd.DataFrame({
    "topic_id": all_topics,
    "group": clusters
})

# 5. 加上语义标签方便查看
topic_labels = topic_model.generate_topic_labels(nr_words=3, separator="_")
id_to_label = {int(l.split("_")[0]): l for l in topic_labels if not l.startswith("-1")}
df_res['label'] = df_res['topic_id'].map(id_to_label)

# 查看最终结果
print(df_res.head(50))

# 检查每个组包含的主题数
print("\n每组主题数量分布：")
print(df_res['group'].value_counts().sort_index())

In [ ]:
# 1. 按照 group 分组，并将 topic_id 聚合为列表
grouped_topics = df_res.groupby('group')['topic_id'].apply(list).reset_index()

# 2. 格式化输出
print(f"共识别出 {len(grouped_topics)} 个主题组：\n")

for _, row in grouped_topics.iterrows():
    # 将 group 编号加 1 方便阅读（从第一组开始计数）
    group_num = row['group'] + 1
    topic_list = row['topic_id']
    
    # 打印格式：第一组：[0, 1, 2, ...]
    print(f"第{group_num}组：{topic_list}")

In [ ]:
# 3. 如果你想同时看到这组主题的关键词标签（方便验证语义一致性）
print("\n" + "="*30 + "\n详细语义分组：\n")
for _, row in grouped_topics.iterrows():
    group_num = row['group'] + 1
    # 获取该组下所有主题的标签
    labels = df_res[df_res['group'] == row['group']]['label'].tolist()
    print(f"第{group_num}组 包含主题：")
    for lbl in labels:
        print(f"  - {lbl}")
    print("-" * 20)

In [ ]:
# 1. 提取嵌套列表格式：[[组1序号], [组2序号], ...]
nested_topic_lists = grouped_topics['topic_id'].tolist()

print("--- 嵌套列表输出 ---")
print(nested_topic_lists)
print("\n" + "="*50 + "\n")

In [ ]:
# 2. 合并后的主题理解方法一：提取每组的代表性关键词（丢失c-tf-idf权重）
# 逻辑：获取每组内所有主题的关键词，计算词频，取前 10 个作为组关键词
group_keywords = {}

for _, row in grouped_topics.iterrows():
    group_num = row['group']
    topics_in_group = row['topic_id']
    
    all_words_in_group = []
    for t_id in topics_in_group:
        # 获取单个主题的关键词 (返回的是 [(word, score), ...])
        words = [word for word, _ in topic_model.get_topic(t_id)]
        all_words_in_group.extend(words)
    
    # 统计词频并取前 10 个最能代表本组的词
    from collections import Counter
    # 过滤掉一些极其常见的重复词，保留每组独特的语义
    most_common_words = [word for word, count in Counter(all_words_in_group).most_common(10)]
    group_keywords[group_num + 1] = most_common_words

# 3. 格式化展示组关键词
print("--- 各组代表性关键词 ---")
for g_id, keywords in group_keywords.items():
    print(f"第{g_id}组共同特征词: {', '.join(keywords)}")

# 合并主题

In [ ]:
# 合并主题后,bertopic_model会改变，如需多次合并，重新调用保存的原始模型
topic_model.merge_topics(docs, nested_topic_lists)
#topic_model.get_topic_info()

In [ ]:
# 获取所有主题的字典 {topic_id: [(word, score), ...]}
all_topics_dict = topic_model.get_topics()
# 过滤掉 -1 (噪声)，并将每个词列表转为逗号连接的字符串
clean_topics_dict = {
    topic_id: ",".join([word for word, score in words_list])
    for topic_id, words_list in all_topics_dict.items()
    if topic_id != -1
}

for item in clean_topics_dict.items():
    print(item)
    
# 检查结果
print(f'已获取{len(clean_topics_dict)}个主题的详细信息，现在通过AI判断主题名称……') 

# AI主题识别

In [ ]:
# AI识别
user_prompt = f"请分析以下主題关键词，并返回 JSON 字典：\n{clean_topics_dict}"

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
# 加载 .env 文件
load_dotenv(".env")
API_KEY=os.environ.get('DEEPSEEK_API_KEY')
deepseek_chat_model = "deepseek-chat" # DeepSeek-V3.2的非思考模式
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com")
# AI识别智能体参数设置：创建专门用于地址推理的 LLM 实例
Deepseek_reasponse = client.chat.completions.create(
    model=deepseek_chat_model,
    messages=[ # 对话消息列表
        {"role": "system", "content": system_prompt}, # 系统提示词，定义助手的行为
        {"role": "user", "content": user_prompt},
    ],
    response_format={'type': 'json_object'}, #强制json格式返回
    stream=False # 非流式响应（一次性返回完整结果）
)
content = json.loads(Deepseek_reasponse.choices[0].message.content)
#content = re.sub(r"^```json\s*|\s*```$", "", content.strip())
# 将键转为整型
formatted_labels = {int(k): v for k, v in content.items()}
print(formatted_labels)
# 使用自定义标签（需要先设置）
topic_model.set_topic_labels(formatted_labels)

In [ ]:
import pandas as pd

h_data = {
        "n_groups": n_groups,
        "all_topics": all_topics # 原始主题列表
    }

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}进一步层次聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_Allset_merged{n_groups}_{year_range}_{data_source}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_bertopic_report(umap_cfg = umap_params, 
                         HDBSCAN_cfg=HDBSCAN_cfg,
                         vectorizer_cfg = vectorizer_params,
                         history= search_history,
                         best_size = best_m_size,
                         model_name = model_name, 
                         output_path = report_name,
                         hierarchical_data = h_data)
# print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=len(formatted_labels),
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=50, r=250, t=100, b=50),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 topic_overtime_merged.html

topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")